In [2]:
import pennylane as qml
from pennylane import numpy as np
np.set_printoptions(precision=2)

## Demo 1: Quantum phase estimation

<img src="fig/qpe-full-screencap.png" width="500px">

In [3]:
T = qml.T.compute_matrix()
X = qml.PauliX.compute_matrix()
I = np.eye(2)

U = np.kron(T, np.kron(X, I))

U

tensor([[0.  +0.j  , 0.  +0.j  , 1.  +0.j  , 0.  +0.j  , 0.  +0.j  ,
         0.  +0.j  , 0.  +0.j  , 0.  +0.j  ],
        [0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 1.  +0.j  , 0.  +0.j  ,
         0.  +0.j  , 0.  +0.j  , 0.  +0.j  ],
        [1.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  ,
         0.  +0.j  , 0.  +0.j  , 0.  +0.j  ],
        [0.  +0.j  , 1.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  ,
         0.  +0.j  , 0.  +0.j  , 0.  +0.j  ],
        [0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  ,
         0.  +0.j  , 0.71+0.71j, 0.  +0.j  ],
        [0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  ,
         0.  +0.j  , 0.  +0.j  , 0.71+0.71j],
        [0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.71+0.71j,
         0.  +0.j  , 0.  +0.j  , 0.  +0.j  ],
        [0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  , 0.  +0.j  ,
         0.71+0.71j, 0.  +0.j  , 0.  +0.j  ]], requires_grad=True)

In [29]:
eigvals, eigvecs = np.linalg.eig(U)

target_eigvec = eigvecs[:, 0]
target_eigval = eigvals[0]

In [30]:
eigvals[7]

(-0.7071067811865476-0.7071067811865475j)

In [31]:
np.angle(target_eigval) / (2 * np.pi) # 001

0.0

In [16]:
target_eigvec

tensor([ 0.00e+00+0.00e+00j,  0.00e+00+0.00e+00j,  0.00e+00+0.00e+00j,
         0.00e+00+0.00e+00j,  0.00e+00+0.00e+00j,  7.07e-01+0.00e+00j,
        -2.16e-17-1.15e-17j, -7.07e-01-1.39e-17j], requires_grad=True)

In [17]:
estimation_wires = [0,1,2]
target_wires = [3,4,5]

dev = qml.device('default.qubit', wires=estimation_wires+target_wires, shots=1)

In [18]:
@qml.qnode(dev)
def qpe_with_template():
    qml.MottonenStatePreparation(target_eigvec, wires=target_wires)
    qml.QuantumPhaseEstimation(
        U,
        target_wires=target_wires,
        estimation_wires=estimation_wires
    )
    return qml.probs(wires=estimation_wires)

In [19]:
qpe_with_template()

tensor([0., 0., 0., 0., 0., 1., 0., 0.], requires_grad=True)

## Demo 2: Order finding

In [24]:
from lecture11_helpers import *

In [25]:
N = 13
a = 5

In [26]:
for exp in range(1, N):
    if (a ** exp) % N == 1:
        print(f"The order of {a} is {exp}")
        break

The order of 5 is 4


In [11]:
U_Na = get_U_Na(a, N)

num_estimation_qubits = 10
num_target_qubits = int(np.log2(len(U_Na)))

estimation_wires = range(num_estimation_qubits)
target_wires = range(num_estimation_qubits, num_estimation_qubits + num_target_qubits)

In [12]:
dev = qml.device('default.qubit', wires=num_estimation_qubits+num_target_qubits, shots=1)

@qml.qnode(dev)
def find_order():

    return qml.sample(wires=estimation_wires)

In [13]:
possible_r = []

for _ in range(10):
    sample = find_order()
    phase = fractional_binary_to_float(sample)
    est_r = phase_to_order(phase, N)
    possible_r.append(est_r)